In [2]:
import time, os, sys, subprocess, shutil
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [3]:
def sp_grep(file, match):
    o = subprocess.run('grep "{}" {}'.format(match, file), check=True, shell=True, stdout=subprocess.PIPE, universal_newlines=True)
    return o.stdout
def epoch_tail(grepopt):
    epochs = [i.split('||')[0].strip()[-2:].strip() for i in grepopt.split('\n') if i != '']
    tails = [i.split(',')[-1].split('.')[0] for i in grepopt.split('\n') if i != '']
    pts = [i.split('CHKPT:')[-1].strip() for i in grepopt.split('\n') if i != '']
    return (epochs, tails, pts)
class FileCopier():
    def __init__(self, prefix):
        self.prefix = prefix

class CheckpointHandler(FileSystemEventHandler):
    def __init__(self, watchDir='', targetDir = ''):
        self.watchDir = watchDir
        self.targetDir = targetDir
        self.prefix = None
        self.log = None
        
    def on_created(self, event):
        if 'log' in event.src_path:
            print("EVENT -- CREATED: {}".format(event.src_path))
            prefix = '.'.join(event.src_path.split('/')[-1].split('.')[:2])
            self.prefix = prefix
            self.log = os.path.join(self.watchDir, prefix+'.log')
        if '.pt' in event.src_path:
            print("EVENT -- CREATED: {}".format(event.src_path))
            o = sp_grep(self.log, match='Epoch')
            es, ts, ps = epoch_tail(grepopt = o)
            print(es, ts, ps)
            for idx, e in enumerate(es):
                if len(e) == 1:
                    e = '0'+e
                psuff = ps[idx]
                if psuff == 'NOUPDATE':
                    continue
                p = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e))
                pt = os.path.join(self.watchDir, self.prefix+psuff)
                tpt = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e), self.prefix+psuff)
                sl = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e), 'xc')
                try:
                    print("os.mkdir({})".format(p))
                    os.mkdir(p)
                    print('waiting...')
                    time.sleep(10)
                except:
                    e = sys.exc_info()[0]
                    print(e)
                    #if directory exists, continue to next file
                    continue
                try:
                    print('waiting...')
                    time.sleep(10)
                    print("shutil.copy:")
                    print("{} -> {}".format(pt, tpt))
                    shutil.copy(src=pt, dst=tpt)
                    print('os.symlink: {} -> {}'.format(tpt, sl))
                    os.symlink(tpt, sl)
                except:
                    e = sys.exc_info()[0]
                    print(e)
                    #either directory exists or file does not yet exist
                    pass

        
    def on_modified(self, event):
        if 'log' in event.src_path and not self.prefix:
            print("EVENT -- MODIFIED: {}".format(event.src_path))
            prefix = '.'.join(event.src_path.split('/')[-1].split('.')[:2])
            self.prefix = prefix
            self.log = os.path.join(self.watchDir, prefix+'.log')
        if '.pt' in event.src_path:
            print("EVENT -- MODIFIED: {}".format(event.src_path))
            o = sp_grep(self.log, match='Epoch')
            es, ts, ps = epoch_tail(grepopt = o)
            print(es, ts, ps)
            for idx, e in enumerate(es):
                if len(e) == 1:
                    e = '0'+e
                psuff = ps[idx]
                if psuff == 'NOUPDATE':
                    continue
                p = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e))
                pt = os.path.join(self.watchDir, self.prefix+psuff)
                tpt = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e), self.prefix+psuff)
                sl = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e), 'xc')
                try:
                    print("os.mkdir({})".format(p))
                    os.mkdir(p)
                    print('waiting...')
                    time.sleep(10)
                except:
                    e = sys.exc_info()[0]
                    print(e)
                    #if directory exists, continue to next file
                    continue
                try:
                    print('waiting...')
                    time.sleep(10)
                    print("shutil.copy:")
                    print("{} -> {}".format(pt, tpt))
                    shutil.copy(src=pt, dst=tpt)
                    print('os.symlink: {} -> {}'.format(tpt, sl))
                    os.symlink(tpt, sl)
                except:
                    e = sys.exc_info()[0]
                    print(e)
                    #either directory exists or file does not yet exist
                    pass 


In [7]:
tst = '/home/awills/Documents/Research/swxcd/train/test_subset2/pts13'
tts = '/2022-08-18_17:46:53.422701'
o = sp_grep(tst+tts+'.log', match='Epoch')
es, ts, ps = epoch_tail(grepopt = o)

In [8]:
es, ts, ps

(['0'], ['|| total loss 42'], ['_0.pt'])

In [6]:
obs = Observer()
watchdir = '/home/awills/Documents/Research/swxcd/train/test_subset2/pts8'
targdir = '/home/awills/Documents/Research/swxcd/eval/test_subset2/pts8'
hand = CheckpointHandler(watchDir = watchdir, targetDir = targdir)
obs.schedule(hand, path=watchdir)
obs.start()

EVENT -- CREATED: /home/awills/Documents/Research/swxcd/train/test_subset2/pts8/2022-08-16_16:22:57.255875.log
EVENT -- CREATED: /home/awills/Documents/Research/swxcd/train/test_subset2/pts8/2022-08-16_16:22:57.255875_current.pt
['0'] ['']
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts8/MODEL_MGGA_e00)
waiting...
waiting...
shutil.copy:
/home/awills/Documents/Research/swxcd/train/test_subset2/pts8/2022-08-16_16:22:57.255875_0.pt -> /home/awills/Documents/Research/swxcd/eval/test_subset2/pts8/MODEL_MGGA_e00/2022-08-16_16:22:57.255875_0.pt
os.symlink: /home/awills/Documents/Research/swxcd/eval/test_subset2/pts8/MODEL_MGGA_e00/2022-08-16_16:22:57.255875_0.pt -> /home/awills/Documents/Research/swxcd/eval/test_subset2/pts8/MODEL_MGGA_e00/xc
EVENT -- MODIFIED: /home/awills/Documents/Research/swxcd/train/test_subset2/pts8/2022-08-16_16:22:57.255875_current.pt
['0'] ['']
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts8/MODEL_MGGA_e00)
<class 'FileExis

In [5]:
obs.stop()
obs.join()